In [ ]:
# Helper functions
def calculate_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

def condition_abc_margin(x):
    if x <= 0.03:
        return 'C'
    elif x > 0.03 and x <= 0.07:
        return 'B'
    else:
        return 'A'

def condition_abc_rev(x):
    if x > 0 and x <= 0.80:
        return "A"
    elif x > 0.80 and x <= 0.95:
        return "B"
    else:
        return 'C'

def condition_xyz(x):
    if x <= 0.5:
        return 'X'
    elif x > 0.5 and x <= 1:
        return 'Y'
    else:
        return 'Z'

In [ ]:
# ABC Analysis
def abc_analysis(df):
    abc = df[['Territory', 'Delivery Note', 'Catalog Name', 'Inventory Name', 'Revenue (VAT Excl.)', 
              'Adjusted Front Margin', 'Adjusted Front Margin(%)']]
    
    # ABC Margin Analysis
    abc_df_mode_with_ter = abc.groupby(['Territory', 'Inventory Name']).agg(
        Product_Margin=('Adjusted Front Margin(%)', calculate_mode),  
        Revenue=('Revenue (VAT Excl.)', 'sum')
    ).reset_index()
    abc_df_mode_with_ter['ABC_Margin_mode'] = abc_df_mode_with_ter['Product_Margin'].apply(condition_abc_margin)

    # ABC Revenue Analysis
    abc_rev_with_ter = abc.groupby(['Territory', 'Inventory Name']).agg(
        Total_revenue=('Revenue (VAT Excl.)', 'sum')).reset_index()
    
    abc_rev_with_ter['Total_revenue_sum'] = abc_rev_with_ter.groupby('Territory')['Total_revenue'].transform('sum')
    abc_rev_with_ter = abc_rev_with_ter.sort_values(by=['Territory', 'Total_revenue'], ascending=[True, False])
    abc_rev_with_ter['Cumulative_sum'] = abc_rev_with_ter.groupby('Territory')['Total_revenue'].cumsum()
    abc_rev_with_ter['sku_rev_percent'] = abc_rev_with_ter['Cumulative_sum'] / abc_rev_with_ter['Total_revenue_sum']
    abc_rev_with_ter['ABC_revenue'] = abc_rev_with_ter['sku_rev_percent'].apply(condition_abc_rev)

    # Merge ABC Revenue and Margin datasets
    final_ter = abc_rev_with_ter.merge(abc_df_mode_with_ter, on=['Territory', 'Inventory Name'], how='left')
    final_ter['ABC(Rev-Mar)'] = final_ter['ABC_revenue'].astype(str) + final_ter['ABC_Margin_mode'].astype(str)

    return final_ter[['Territory', 'Inventory Name', 'Total_revenue', 'ABC_revenue', 'Product_Margin', 
                      'ABC_Margin_mode', 'ABC(Rev-Mar)']]

# XYZ Analysis
def xyz_analysis(df):
    xyz = df[['Territory', 'DN Delivery Date', 'Inventory Name', 'Revenue (VAT Excl.)']]
    xyz['year'] = xyz['DN Delivery Date'].dt.year
    xyz['month'] = xyz['DN Delivery Date'].dt.month
    xyz['Year_Month'] = xyz['month'].astype(str) + '-' + xyz['year'].astype(str)

    xyz_df_ter = xyz.groupby(['Territory', 'Inventory Name', 'Year_Month'])['Revenue (VAT Excl.)'].sum().reset_index()
    xyz_df_ter = xyz_df_ter.pivot(index=['Territory', 'Inventory Name'], columns='Year_Month', 
                                   values='Revenue (VAT Excl.)').reset_index().fillna(0)

    # Identify sales columns dynamically
    sales_columns = [col for col in xyz_df_ter.columns if col not in ['Territory', 'Inventory Name']]

    # Calculate total revenue, average revenue, and standard deviation dynamically
    xyz_df_ter['total_revenue'] = xyz_df_ter[sales_columns].sum(axis=1, numeric_only=True)
    num_months = len(sales_columns)
    xyz_df_ter['average_revenue'] = xyz_df_ter['total_revenue'] / num_months if num_months > 0 else 0
    xyz_df_ter['std_dev'] = xyz_df_ter[sales_columns].std(axis=1, numeric_only=True)

    # Calculate Coefficient of Variance (CoV)
    xyz_df_ter['CoV'] = xyz_df_ter['std_dev'] / xyz_df_ter['average_revenue']
    xyz_df_ter['Territory_XYZ'] = xyz_df_ter['CoV'].apply(condition_xyz)

    return xyz_df_ter[['Territory', 'Inventory Name', 'total_revenue', 'CoV', 'std_dev', 'average_revenue', 'Territory_XYZ']]

# ABC-XYZ Merging
def merge_abc_xyz(territory_abc_df, xyz_df):
    abc_xyz = territory_abc_df.merge(xyz_df, on=['Territory', 'Inventory Name'], how='left')
    abc_xyz['ABC_XYZ'] = abc_xyz['ABC(Rev-Mar)'].astype(str) + abc_xyz['Territory_XYZ'].astype(str)
    return abc_xyz

# Main function to execute all analyses
def main(filepath):
    df = load_and_preprocess_data(filepath)
    territory_abc_df = abc_analysis(df)
    xyz_df = xyz_analysis(df)
    abc_xyz = merge_abc_xyz(territory_abc_df, xyz_df)
    
    return abc_xyz